# {Breaking the Glass Firewall: Why Women Leave Tech Careers and Why Those Who Stay Don’t Advance}📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->

This project explores why women in technology are more likely to experience limited career progression and leave the industry at higher rates than their male counterparts. Understanding these differences in career trajectories between men and women is essential for promoting fairness in the workplace, reducing costs associated with turnover, and improving overall organizational success by retaining diverse talent.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->

How do promotion and retention rates for women compare to those for men at similar career stages in the tech industry?

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->

**Hypothesis:**  
Women in technology experience lower promotion rates and leave the industry at higher rates compared to their male counterparts, even when they have similar qualifications and experience. This disparity is driven by factors that disproportionately affect women, including a higher likelihood of layoffs and gender-based discrimination. As a result, women are less likely to reach senior positions or remain in the tech industry long-term.


## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->

**Data Sources**  
*HackerRank 2018 Developer Survey (Kaggle):* Published in 2018, based on 2017 survey responses.  
*Pew Research Center 2017 STEM Survey (zip file):* Based on 2017 responses.  
*NSF's National Center for Science and Engineering Statistics (Web-Scraped Tables):* Spans several years, ending in 2019, with specific data points from 2017 for comparison.

**Relating the Data**  
The datasets can be linked based on the shared year (2017) and gender as a common variable. Gender will serve as a primary key or part of a composite key for linking.

## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->

After joining the data from the various sources, I will generate specific visualizations to confirm or reject my hypothesis.

**Planned Visualizations to Support the Hypothesis:**
1. *Line Chart: Gender and Age Distribution in Technology*  
A line chart will display the number of employees in the technology sector, separated by gender and age groups. The x-axis will represent the age groups (e.g., 20-25, 26-30, etc.), and the y-axis will show the number of employees. This chart will highlight the drop-off point where women begin to leave the industry earlier than men, helping visualize retention issues.

2. *100% Stacked Column Chart: Men vs Women in Different Tech Roles*  
A 100% stacked column chart will show the proportional representation of men and women across different tech roles (e.g., Junior Developers, Senior Developers, Managers, Executives). Each column will represent a different role, and the stacked columns will show the gender distribution within that role as a percentage. This will provide a clear visual of how underrepresented women are in higher-level positions.

3. *Side-by-Side Column Chart: Workplace Concerns for Men vs Women*  
A side-by-side column chart will compare the key workplace concerns between men and women, such as issues with career progression, work-life balance, pay disparity, and workplace discrimination. Each concern will have two columns—one representing men and one representing women. This will make it easy to see where concerns overlap and where significant differences exist between the genders.

#### Package Imports

In [80]:
#import packages
import os # to create subfolder for data organization
from dotenv import load_dotenv
load_dotenv(override=True)

import opendatasets as od
import pandas as pd
import requests
import re # for string manipulation

import matplotlib.pyplot as plt
import seaborn as sns

from zipfile import ZipFile
from urllib.request import urlretrieve
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix


#### Import Dataset 1:

In [81]:
# import dataset from Kaggle using URL 
dataset_url = "https://www.kaggle.com/datasets/hackerrank/developer-survey-2018/data"
od.download(dataset_url, data_dir="./data")

Dataset URL: https://www.kaggle.com/datasets/hackerrank/developer-survey-2018


100%|██████████| 4.81M/4.81M [00:00<00:00, 10.1MB/s]


#### Convert dataset to a pandas dataframe and inspect data

In [82]:
# Define the data directory
data_dir = './data/developer-survey-2018'

# Read each CSV file into its own DataFrame with meaningful names
country_code_df = pd.read_csv(os.path.join(data_dir, 'Country-Code-Mapping.csv'))
dev_survey_codebook_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Codebook.csv'))
dev_survey_numeric_mapping_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Numeric-Mapping.csv'))
dev_survey_numeric_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Numeric.csv'))
dev_survey_values_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Values.csv'))

# Display the first 5 records from each DataFrame
display(country_code_df.head(5))
display(dev_survey_codebook_df.head(5))
display(dev_survey_numeric_mapping_df.head(5))
display(dev_survey_numeric_df.head(5))
display(dev_survey_values_df.head(5))

C:\Users\Michelle\AppData\Local\Temp\ipykernel_7980\3349565919.py:8: DtypeWarning: Columns (10,19,137,138,250) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_survey_numeric_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Numeric.csv'))
C:\Users\Michelle\AppData\Local\Temp\ipykernel_7980\3349565919.py:9: DtypeWarning: Columns (3,10,19,137,138) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_survey_values_df = pd.read_csv(os.path.join(data_dir, 'HackerRank-Developer-Survey-2018-Values.csv'))


,Value,Label
0,4,Afghanistan
1,6,Albania
2,7,Algeria
3,8,American Samoa
4,9,Andorra


,Data Field,Survey Question,Notes
0,RespondentID,NaN,Respondent ID
1,StartDate,NaN,When did they start (date and time)
2,EndDate,NaN,When did they end (date and time)
3,CountryNumeric2,NaN,see Country-Code-Mapping.csv
4,q1AgeBeginCoding,At what age did you start coding,NaN


,Data Field,Value,Label
0,q1AgeBeginCoding,1,5 - 10 years old
1,q1AgeBeginCoding,2,11 - 15 years old
2,q1AgeBeginCoding,3,16 - 20 years old
3,q1AgeBeginCoding,4,21 - 25 years old
4,q1AgeBeginCoding,5,26 - 30 years old


,RespondentID,StartDate,EndDate,CountryNumeric2,q1AgeBeginCoding,q2Age,q3Gender,q4Education,q0004_other,q5DegreeFocus,...,q30LearnCodeOther,q0030_other,q31Level3,q32RecommendHackerRank,q0032_other,q33HackerRankChallforJob,q34PositiveExp,q34IdealLengHackerRankTest,q0035_other,q36Level4
0,6464453728,10/19/17 11:51,10/20/17 12:05,148.0,3,3,2,3,NaN,1,...,1,datacamp,1,1,NaN,2,NaN,#NULL!,NaN,2
1,6478031510,10/26/17 6:18,10/26/17 7:49,164.0,3,4,1,7,NaN,2,...,0,NaN,1,1,NaN,2,NaN,#NULL!,NaN,2
2,6464392829,10/19/17 10:44,10/19/17 10:56,98.0,2,2,2,3,NaN,2,...,0,NaN,1,1,NaN,2,NaN,#NULL!,NaN,2
3,6481629912,10/27/17 1:51,10/27/17 2:05,43.0,2,2,1,5,NaN,1,...,0,NaN,1,1,NaN,2,NaN,#NULL!,NaN,3
4,6488385057,10/31/17 11:46,10/31/17 11:59,NaN,3,4,2,5,NaN,0,...,1,Blogs/articles by industry leaders,1,1,NaN,2,NaN,#NULL!,NaN,3


,RespondentID,StartDate,EndDate,CountryNumeric2,q1AgeBeginCoding,q2Age,q3Gender,q4Education,q0004_other,q5DegreeFocus,...,q30LearnCodeOther,q0030_other,q31Level3,q32RecommendHackerRank,q0032_other,q33HackerRankChallforJob,q34PositiveExp,q34IdealLengHackerRankTest,q0035_other,q36Level4
0,6464453728,10/19/17 11:51,10/20/17 12:05,South Korea,16 - 20 years old,18 - 24 years old,Female,Some college,NaN,Computer Science,...,Other (please specify),datacamp,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue
1,6478031510,10/26/17 6:18,10/26/17 7:49,Ukraine,16 - 20 years old,25 - 34 years old,Male,"Post graduate degree (Masters, PhD)",NaN,"Other STEM (science, technology, engineering, ...",...,NaN,NaN,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue
2,6464392829,10/19/17 10:44,10/19/17 10:56,Malaysia,11 - 15 years old,12 - 18 years old,Female,Some college,NaN,"Other STEM (science, technology, engineering, ...",...,NaN,NaN,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue
3,6481629912,10/27/17 1:51,10/27/17 2:05,Curaçao,11 - 15 years old,12 - 18 years old,Male,College graduate,NaN,Computer Science,...,NaN,NaN,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Hashmap
4,6488385057,10/31/17 11:46,10/31/17 11:59,NaN,16 - 20 years old,25 - 34 years old,Female,College graduate,NaN,NaN,...,Other (please specify),Blogs/articles by industry leaders,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Hashmap


In [83]:
# Find United States in the country_code_df to use for filtering purposes

# Rename columns
country_code_df.rename(columns={'Value': 'country_code', 'Label': 'country'}, inplace=True)

# Filter for the United States
us_country_code_df = country_code_df[country_code_df['country'] == 'United States']

# Display the resulting DataFrame
display(us_country_code_df)

,country_code,country
145,167,United States


In [84]:
# Filter the DataFrame for United States questionnaire responses
us_dev_survey_numeric_df = dev_survey_numeric_df[dev_survey_numeric_df['CountryNumeric2'] == 167]

# Display the number of records
num_records_numeric = us_dev_survey_numeric_df.shape[0]
display(f"Number of records: {num_records_numeric}")

# Display the resulting DataFrame
display(us_dev_survey_numeric_df.head(5))

'Number of records: 4937'

,RespondentID,StartDate,EndDate,CountryNumeric2,q1AgeBeginCoding,q2Age,q3Gender,q4Education,q0004_other,q5DegreeFocus,...,q30LearnCodeOther,q0030_other,q31Level3,q32RecommendHackerRank,q0032_other,q33HackerRankChallforJob,q34PositiveExp,q34IdealLengHackerRankTest,q0035_other,q36Level4
5,6463843138,10/19/17 3:02,10/19/17 3:18,167.0,8,5,1,5,NaN,1,...,1,SoloLearn,1,1,NaN,2,NaN,#NULL!,NaN,2
6,6458326054,10/17/17 3:18,10/17/17 3:33,167.0,3,6,1,7,NaN,1,...,0,NaN,1,1,NaN,1,1.0,4,NaN,2
7,6467198274,10/21/17 8:55,10/21/17 9:06,167.0,3,3,1,5,NaN,2,...,0,NaN,1,1,NaN,2,NaN,#NULL!,NaN,1
24,6460870080,10/18/17 1:18,10/18/17 1:25,167.0,3,3,1,5,NaN,2,...,0,NaN,1,1,NaN,2,NaN,#NULL!,NaN,2
40,6479693460,10/26/17 7:10,10/26/17 7:20,167.0,2,6,1,5,NaN,1,...,1,IRC,1,1,NaN,2,NaN,#NULL!,NaN,2


In [85]:
# Reduce dataframe columns to only those relevant to supporting or disproving my hypothesis

# List of relevant columns to keep
columns_to_keep = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole',
    'q12JobCritPrefTechStack', 'q12JobCritCompMission',
    'q12JobCritCompCulture', 'q12JobCritWorkLifeBal',
    'q12JobCritCompensation', 'q12JobCritProximity',
    'q12JobCritPerks', 'q12JobCritSmartPeopleTeam',
    'q12JobCritImpactwithProduct', 'q12JobCritInterestProblems',
    'q12JobCritFundingandValuation', 'q12JobCritStability',
    'q12JobCritProfGrowth', 'q16HiringManager',
    'q17HirChaNoDiversCandidates', 'q20CandYearExp',
    'q20CandCompScienceDegree', 'q20CandCodingBootcamp',
    'q20CandSkillCert', 'q20CandHackerRankActivity',
    'q20CandOtherCodingCommAct', 'q20CandGithubPersProj',
    'q20CandOpenSourceContrib', 'q20CandHackathonPart',
    'q20CandPrevWorkExp', 'q20CandPrestigeDegree',
    'q20CandLinkInSkills', 'q20CandGithubPersProj2'
]

# Create a new DataFrame with only the selected columns
filtered_us_dev_survey_numeric_df = us_dev_survey_numeric_df[columns_to_keep]

# Display information about the DataFrame 
filtered_us_dev_survey_numeric_df.info()

# Display the resulting DataFrame
display(filtered_us_dev_survey_numeric_df.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 4937 entries, 5 to 9892
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   RespondentID                   4937 non-null   int64 
 1   q2Age                          4937 non-null   object
 2   q3Gender                       4937 non-null   object
 3   q10Industry                    4937 non-null   object
 4   q8JobLevel                     4937 non-null   int64 
 5   q9CurrentRole                  4937 non-null   object
 6   q12JobCritPrefTechStack        4937 non-null   int64 
 7   q12JobCritCompMission          4937 non-null   int64 
 8   q12JobCritCompCulture          4937 non-null   int64 
 9   q12JobCritWorkLifeBal          4937 non-null   int64 
 10  q12JobCritCompensation         4937 non-null   int64 
 11  q12JobCritProximity            4937 non-null   int64 
 12  q12JobCritPerks                4937 non-null   int64 
 13  q12JobCr

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,...,q20CandSkillCert,q20CandHackerRankActivity,q20CandOtherCodingCommAct,q20CandGithubPersProj,q20CandOpenSourceContrib,q20CandHackathonPart,q20CandPrevWorkExp,q20CandPrestigeDegree,q20CandLinkInSkills,q20CandGithubPersProj2
5,6463843138,5,1,14,0,0,0,0,1,1,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
6,6458326054,6,1,5,9,19,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
7,6467198274,3,1,4,2,0,0,0,1,0,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
24,6460870080,3,1,0,1,18,0,0,0,1,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
40,6479693460,6,1,14,10,11,1,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [86]:
# Define columns for Worker Responses Numeric DataFrame
worker_columns_numeric = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole',
    'q12JobCritPrefTechStack', 'q12JobCritCompMission',
    'q12JobCritCompCulture', 'q12JobCritWorkLifeBal',
    'q12JobCritCompensation', 'q12JobCritProximity',
    'q12JobCritPerks', 'q12JobCritSmartPeopleTeam',
    'q12JobCritImpactwithProduct', 'q12JobCritInterestProblems',
    'q12JobCritFundingandValuation', 'q12JobCritStability',
    'q12JobCritProfGrowth', 'q16HiringManager'
]

# Create Worker Responses Numeric DataFrame
worker_responses_numeric_df = filtered_us_dev_survey_numeric_df[
    filtered_us_dev_survey_numeric_df['q16HiringManager'] == 2  # Not a hiring manager
][worker_columns_numeric]

# Define columns for Hiring Manager Responses Numeric DataFrame
hiring_manager_columns_numeric = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole', 'q16HiringManager',
    'q17HirChaNoDiversCandidates', 'q20CandYearExp',
    'q20CandCompScienceDegree', 'q20CandCodingBootcamp',
    'q20CandSkillCert', 'q20CandHackerRankActivity',
    'q20CandOtherCodingCommAct', 'q20CandGithubPersProj',
    'q20CandOpenSourceContrib', 'q20CandHackathonPart',
    'q20CandPrevWorkExp', 'q20CandPrestigeDegree',
    'q20CandLinkInSkills', 'q20CandGithubPersProj2'
]

# Create Hiring Manager Responses Numeric DataFrame
hiring_manager_responses_numeric_df = filtered_us_dev_survey_numeric_df[
    filtered_us_dev_survey_numeric_df['q16HiringManager'] == 1  # Is a hiring manager
][hiring_manager_columns_numeric]

# Display the resulting Worker Responses Numeric DataFrame and info about the dataframe
print("Worker Responses Numeric DataFrame:")
display(worker_responses_numeric_df.info())
display(worker_responses_numeric_df.head(5))

# Display the resulting Hiring Manager Responses Numeric DataFrame and info about the dataframe
print("Hiring Manager Responses Numeric DataFrame:")
display(hiring_manager_responses_numeric_df.info())
display(hiring_manager_responses_numeric_df.head(5))


Worker Responses Numeric DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 3289 entries, 5 to 9892
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   RespondentID                   3289 non-null   int64 
 1   q2Age                          3289 non-null   object
 2   q3Gender                       3289 non-null   object
 3   q10Industry                    3289 non-null   object
 4   q8JobLevel                     3289 non-null   int64 
 5   q9CurrentRole                  3289 non-null   object
 6   q12JobCritPrefTechStack        3289 non-null   int64 
 7   q12JobCritCompMission          3289 non-null   int64 
 8   q12JobCritCompCulture          3289 non-null   int64 
 9   q12JobCritWorkLifeBal          3289 non-null   int64 
 10  q12JobCritCompensation         3289 non-null   int64 
 11  q12JobCritProximity            3289 non-null   int64 
 12  q12JobCritPerks                

None

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,q12JobCritCompensation,q12JobCritProximity,q12JobCritPerks,q12JobCritSmartPeopleTeam,q12JobCritImpactwithProduct,q12JobCritInterestProblems,q12JobCritFundingandValuation,q12JobCritStability,q12JobCritProfGrowth,q16HiringManager
5,6463843138,5,1,14,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2
7,6467198274,3,1,4,2,0,0,0,1,0,0,0,0,1,0,0,0,0,1,2
24,6460870080,3,1,0,1,18,0,0,0,1,1,1,0,0,0,0,0,0,0,2
44,6464279786,5,1,14,6,3,0,0,0,1,1,0,0,1,0,0,0,0,0,2
45,6463574850,2,1,#NULL!,1,18,0,0,0,0,0,0,0,0,0,0,0,0,0,2


Hiring Manager Responses Numeric DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1648 entries, 6 to 9890
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   RespondentID                 1648 non-null   int64 
 1   q2Age                        1648 non-null   object
 2   q3Gender                     1648 non-null   object
 3   q10Industry                  1648 non-null   object
 4   q8JobLevel                   1648 non-null   int64 
 5   q9CurrentRole                1648 non-null   object
 6   q16HiringManager             1648 non-null   int64 
 7   q17HirChaNoDiversCandidates  1648 non-null   object
 8   q20CandYearExp               1648 non-null   object
 9   q20CandCompScienceDegree     1648 non-null   object
 10  q20CandCodingBootcamp        1648 non-null   object
 11  q20CandSkillCert             1648 non-null   object
 12  q20CandHackerRankActivity    1648 non-null   object

None

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q16HiringManager,q17HirChaNoDiversCandidates,q20CandYearExp,q20CandCompScienceDegree,...,q20CandSkillCert,q20CandHackerRankActivity,q20CandOtherCodingCommAct,q20CandGithubPersProj,q20CandOpenSourceContrib,q20CandHackathonPart,q20CandPrevWorkExp,q20CandPrestigeDegree,q20CandLinkInSkills,q20CandGithubPersProj2
6,6458326054,6,1,5,9,19,1,0,1,0,...,0,0,0,0,0,0,1,1,0,0
40,6479693460,6,1,14,10,11,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1
42,6463550038,4,1,5,5,9,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
43,6480657500,4,1,11,6,11,1,1,1,1,...,1,0,0,0,0,0,1,0,0,0
47,6478832462,4,2,9,5,4,1,0,1,0,...,0,0,0,0,0,0,1,1,0,0


In [87]:
# Filter the DataFrame for CountryNumeric2 = "United States"
us_dev_survey_values_df = dev_survey_values_df[dev_survey_values_df['CountryNumeric2'] == "United States"]

# Display the number of records
num_records_values = us_dev_survey_values_df.shape[0]
display(f"Number of records: {num_records_values}")

# Display the resulting DataFrame
display(us_dev_survey_values_df.head(5))

'Number of records: 4937'

,RespondentID,StartDate,EndDate,CountryNumeric2,q1AgeBeginCoding,q2Age,q3Gender,q4Education,q0004_other,q5DegreeFocus,...,q30LearnCodeOther,q0030_other,q31Level3,q32RecommendHackerRank,q0032_other,q33HackerRankChallforJob,q34PositiveExp,q34IdealLengHackerRankTest,q0035_other,q36Level4
5,6463843138,10/19/17 3:02,10/19/17 3:18,United States,41 - 50 years old,35 - 44 years old,Male,College graduate,NaN,Computer Science,...,Other (please specify),SoloLearn,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue
6,6458326054,10/17/17 3:18,10/17/17 3:33,United States,16 - 20 years old,45 - 54 years old,Male,"Post graduate degree (Masters, PhD)",NaN,Computer Science,...,NaN,NaN,num%2 == 0,Yes,NaN,Yes,1.0,2 to 4 hours,NaN,Queue
7,6467198274,10/21/17 8:55,10/21/17 9:06,United States,16 - 20 years old,18 - 24 years old,Male,College graduate,NaN,"Other STEM (science, technology, engineering, ...",...,NaN,NaN,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Set
24,6460870080,10/18/17 1:18,10/18/17 1:25,United States,16 - 20 years old,18 - 24 years old,Male,College graduate,NaN,"Other STEM (science, technology, engineering, ...",...,NaN,NaN,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue
40,6479693460,10/26/17 7:10,10/26/17 7:20,United States,11 - 15 years old,45 - 54 years old,Male,College graduate,NaN,Computer Science,...,Other (please specify),IRC,num%2 == 0,Yes,NaN,No,NaN,#NULL!,NaN,Queue


In [88]:
# Reduce dataframe columns to only those relevant to supporting or disproving my hypothesis

# List of relevant columns to keep
columns_to_keep = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole',
    'q12JobCritPrefTechStack', 'q12JobCritCompMission',
    'q12JobCritCompCulture', 'q12JobCritWorkLifeBal',
    'q12JobCritCompensation', 'q12JobCritProximity',
    'q12JobCritPerks', 'q12JobCritSmartPeopleTeam',
    'q12JobCritImpactwithProduct', 'q12JobCritInterestProblems',
    'q12JobCritFundingandValuation', 'q12JobCritStability',
    'q12JobCritProfGrowth', 'q16HiringManager',
    'q17HirChaNoDiversCandidates', 'q20CandYearExp',
    'q20CandCompScienceDegree', 'q20CandCodingBootcamp',
    'q20CandSkillCert', 'q20CandHackerRankActivity',
    'q20CandOtherCodingCommAct', 'q20CandGithubPersProj',
    'q20CandOpenSourceContrib', 'q20CandHackathonPart',
    'q20CandPrevWorkExp', 'q20CandPrestigeDegree',
    'q20CandLinkInSkills', 'q20CandGithubPersProj2'
]

# Create a new DataFrame with only the selected columns
filtered_us_dev_survey_values_df = us_dev_survey_values_df[columns_to_keep]

# Display information about the DataFrame 
filtered_us_dev_survey_values_df.info()

# Display the resulting DataFrame
display(filtered_us_dev_survey_values_df.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 4937 entries, 5 to 9892
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   RespondentID                   4937 non-null   int64 
 1   q2Age                          4937 non-null   object
 2   q3Gender                       4937 non-null   object
 3   q10Industry                    4416 non-null   object
 4   q8JobLevel                     4661 non-null   object
 5   q9CurrentRole                  4666 non-null   object
 6   q12JobCritPrefTechStack        971 non-null    object
 7   q12JobCritCompMission          743 non-null    object
 8   q12JobCritCompCulture          2095 non-null   object
 9   q12JobCritWorkLifeBal          2688 non-null   object
 10  q12JobCritCompensation         2342 non-null   object
 11  q12JobCritProximity            860 non-null    object
 12  q12JobCritPerks                488 non-null    object
 13  q12JobCr

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,...,q20CandSkillCert,q20CandHackerRankActivity,q20CandOtherCodingCommAct,q20CandGithubPersProj,q20CandOpenSourceContrib,q20CandHackathonPart,q20CandPrevWorkExp,q20CandPrestigeDegree,q20CandLinkInSkills,q20CandGithubPersProj2
5,6463843138,35 - 44 years old,Male,Technology,NaN,NaN,NaN,NaN,Company culture,Good work/life balance,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
6,6458326054,45 - 54 years old,Male,Financial Services,Director / VP of Engineering,Unemployed,NaN,Company mission,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,Prestige of degree,NaN,NaN
7,6467198274,18 - 24 years old,Male,Education,New grad,NaN,NaN,NaN,Company culture,NaN,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
24,6460870080,18 - 24 years old,Male,NaN,Student,Student,NaN,NaN,NaN,Good work/life balance,...,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
40,6479693460,45 - 54 years old,Male,Technology,Founder / CEO / CTO,Software Architect,Preferred tech stack,NaN,Company culture,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,NaN,NaN,Github or personal projects


In [89]:
# Define columns for Worker Responses Values DataFrame
worker_columns_values = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole', 'q16HiringManager',
    'q12JobCritPrefTechStack', 'q12JobCritCompMission',
    'q12JobCritCompCulture', 'q12JobCritWorkLifeBal',
    'q12JobCritCompensation', 'q12JobCritProximity',
    'q12JobCritPerks', 'q12JobCritSmartPeopleTeam',
    'q12JobCritImpactwithProduct', 'q12JobCritInterestProblems',
    'q12JobCritFundingandValuation', 'q12JobCritStability',
    'q12JobCritProfGrowth'
]

# Create Worker Responses Values DataFrame
worker_responses_values_df = filtered_us_dev_survey_values_df[
    filtered_us_dev_survey_values_df['q16HiringManager'] == 'No'  # Not a hiring manager
][worker_columns_values]

# Define columns for Hiring Manager Responses Values DataFrame
hiring_manager_columns_values = [
    'RespondentID', 'q2Age', 'q3Gender', 'q10Industry',
    'q8JobLevel', 'q9CurrentRole', 'q16HiringManager',
    'q17HirChaNoDiversCandidates', 'q20CandYearExp',
    'q20CandCompScienceDegree', 'q20CandCodingBootcamp',
    'q20CandSkillCert', 'q20CandHackerRankActivity',
    'q20CandOtherCodingCommAct', 'q20CandGithubPersProj',
    'q20CandOpenSourceContrib', 'q20CandHackathonPart',
    'q20CandPrevWorkExp', 'q20CandPrestigeDegree',
    'q20CandLinkInSkills', 'q20CandGithubPersProj2'
]

# Create Hiring Manager Responses Values DataFrame
hiring_manager_responses_values_df = filtered_us_dev_survey_values_df[
    filtered_us_dev_survey_values_df['q16HiringManager'] == 'Yes'  # Is a hiring manager
][hiring_manager_columns_values]

# Display the resulting Worker Responses Values DataFrame and info about the dataframe
print("Worker Responses Values DataFrame:")
display(worker_responses_values_df.info())
display(worker_responses_values_df.head(5))

# Display the resulting Hiring Manager Responses Values DataFrame and info about the dataframe
print("Hiring Manager Responses Values DataFrame:")
display(hiring_manager_responses_values_df.info())
display(hiring_manager_responses_values_df.head(5))

Worker Responses Values DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 3289 entries, 5 to 9892
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   RespondentID                   3289 non-null   int64 
 1   q2Age                          3289 non-null   object
 2   q3Gender                       3289 non-null   object
 3   q10Industry                    2847 non-null   object
 4   q8JobLevel                     3125 non-null   object
 5   q9CurrentRole                  3125 non-null   object
 6   q16HiringManager               3289 non-null   object
 7   q12JobCritPrefTechStack        597 non-null    object
 8   q12JobCritCompMission          517 non-null    object
 9   q12JobCritCompCulture          1430 non-null   object
 10  q12JobCritWorkLifeBal          1834 non-null   object
 11  q12JobCritCompensation         1401 non-null   object
 12  q12JobCritProximity            5

None

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q16HiringManager,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,q12JobCritCompensation,q12JobCritProximity,q12JobCritPerks,q12JobCritSmartPeopleTeam,q12JobCritImpactwithProduct,q12JobCritInterestProblems,q12JobCritFundingandValuation,q12JobCritStability,q12JobCritProfGrowth
5,6463843138,35 - 44 years old,Male,Technology,NaN,NaN,No,NaN,NaN,Company culture,Good work/life balance,Compensation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6467198274,18 - 24 years old,Male,Education,New grad,NaN,No,NaN,NaN,Company culture,NaN,NaN,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,Professional growth and learning
24,6460870080,18 - 24 years old,Male,NaN,Student,Student,No,NaN,NaN,NaN,Good work/life balance,Compensation,Proximity to where you live,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,6464279786,35 - 44 years old,Male,Technology,Principal engineer,Back-end Developer,No,NaN,NaN,NaN,Good work/life balance,Compensation,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,NaN
45,6463574850,12 - 18 years old,Male,#NULL!,Student,Student,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Hiring Manager Responses Values DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1648 entries, 6 to 9890
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   RespondentID                 1648 non-null   int64 
 1   q2Age                        1648 non-null   object
 2   q3Gender                     1648 non-null   object
 3   q10Industry                  1569 non-null   object
 4   q8JobLevel                   1536 non-null   object
 5   q9CurrentRole                1541 non-null   object
 6   q16HiringManager             1648 non-null   object
 7   q17HirChaNoDiversCandidates  432 non-null    object
 8   q20CandYearExp               1054 non-null   object
 9   q20CandCompScienceDegree     706 non-null    object
 10  q20CandCodingBootcamp        67 non-null     object
 11  q20CandSkillCert             189 non-null    object
 12  q20CandHackerRankActivity    74 non-null     object


None

,RespondentID,q2Age,q3Gender,q10Industry,q8JobLevel,q9CurrentRole,q16HiringManager,q17HirChaNoDiversCandidates,q20CandYearExp,q20CandCompScienceDegree,...,q20CandSkillCert,q20CandHackerRankActivity,q20CandOtherCodingCommAct,q20CandGithubPersProj,q20CandOpenSourceContrib,q20CandHackathonPart,q20CandPrevWorkExp,q20CandPrestigeDegree,q20CandLinkInSkills,q20CandGithubPersProj2
6,6458326054,45 - 54 years old,Male,Financial Services,Director / VP of Engineering,Unemployed,Yes,NaN,Years of experience,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,Prestige of degree,NaN,NaN
40,6479693460,45 - 54 years old,Male,Technology,Founder / CEO / CTO,Software Architect,Yes,Not enough diverse candidates,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,NaN,NaN,Github or personal projects
42,6463550038,25 - 34 years old,Male,Financial Services,Senior developer,Software Engineer,Yes,NaN,Years of experience,Computer science degree or related,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,NaN,NaN,NaN
43,6480657500,25 - 34 years old,Male,Non-Profit,Principal engineer,Software Architect,Yes,Not enough diverse candidates,Years of experience,Computer science degree or related,...,Skill Certifications,NaN,NaN,NaN,NaN,NaN,Previous work experience,NaN,NaN,NaN
47,6478832462,25 - 34 years old,Female,Insurance,Senior developer,Full-stack Developer,Yes,NaN,Years of experience,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Previous work experience,Prestige of degree,NaN,NaN


In [101]:
# Clean the workers dataframe

# Rename columns for clarity
worker_responses_values_df.rename(columns={
    'q2Age': 'Age',
    'q3Gender': 'Gender',
    'q10Industry': 'Industry',
    'q8JobLevel': 'Job Level',
    'q9CurrentRole': 'Current Role'
}, inplace=True)

# Drop records where Gender is null
worker_responses_values_df.dropna(subset=['Gender'], inplace=True)

# Drop records where Gender is '#NULL!'
worker_responses_values_df.drop(
    worker_responses_values_df[worker_responses_values_df['Gender'] == '#NULL!'].index,
    inplace=True
)

# Filter out respondents who are categorized as "Under 12 years old" or "12 - 18 years old"
worker_responses_values_df = worker_responses_values_df[
    ~worker_responses_values_df['Age'].isin(["Under 12 years old", "12 - 18 years old"])
]

# Filter out rows where Age is '#NULL!'
worker_responses_values_df = worker_responses_values_df[
    worker_responses_values_df['Age'] != '#NULL!'
]

# Filter out rows where Current Role or Job Level is "Student"
worker_responses_values_df = worker_responses_values_df[
    (worker_responses_values_df['Current Role'] != 'Student') & 
    (worker_responses_values_df['Job Level'] != 'Student')
]

# Filter out rows where both Job Level and Current Role are NaN
worker_responses_values_df = worker_responses_values_df[
    ~ (worker_responses_values_df['Job Level'].isna() & worker_responses_values_df['Current Role'].isna())
]

# Display how many rows remain
print(f"Remaining records after cleaning: {worker_responses_values_df.shape[0]}")

# Display the cleaned DataFrame
display(worker_responses_values_df.head(5))

Remaining records after cleaning: 1460


,RespondentID,Age,Gender,Industry,Job Level,Current Role,q16HiringManager,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,q12JobCritCompensation,q12JobCritProximity,q12JobCritPerks,q12JobCritSmartPeopleTeam,q12JobCritImpactwithProduct,q12JobCritInterestProblems,q12JobCritFundingandValuation,q12JobCritStability,q12JobCritProfGrowth
7,6467198274,18 - 24 years old,Male,Education,New grad,NaN,No,NaN,NaN,Company culture,NaN,NaN,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,Professional growth and learning
44,6464279786,35 - 44 years old,Male,Technology,Principal engineer,Back-end Developer,No,NaN,NaN,NaN,Good work/life balance,Compensation,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,NaN
48,6463324693,25 - 34 years old,Male,Technology,Senior developer,NaN,No,NaN,NaN,NaN,NaN,Compensation,NaN,Perks,NaN,NaN,NaN,NaN,NaN,Professional growth and learning
75,6463564885,25 - 34 years old,Male,Technology,Level 1 developer (junior),Software Engineer,No,NaN,NaN,NaN,NaN,Compensation,NaN,Perks,Smart people / team,NaN,NaN,NaN,NaN,Professional growth and learning
102,6464022754,25 - 34 years old,Male,Media,Level 1 developer (junior),Back-end Developer,No,NaN,NaN,Company culture,Good work/life balance,Compensation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# Review dataset to determine what data is relevant

# Check Current Role for Job Level "New grad"
new_grad_roles = worker_responses_values_df[worker_responses_values_df['Job Level'] == 'New grad'] \
    .groupby('Current Role').size().reset_index(name='Counts')

# Filter for rows where Job Level is NaN and Current Role is not NaN
nan_job_level_current_role_df = worker_responses_values_df[worker_responses_values_df['Job Level'].isna() & 
    worker_responses_values_df['Current Role'].notna()]

# Display results
print("Current Roles for Job Level 'New grad':")
display(new_grad_roles)

print("Rows where Job Level is NaN and Current Role is not NaN:")
display(nan_job_level_current_role_df[['Job Level', 'Current Role']])


Current Roles for Job Level 'New grad':


,Current Role,Counts
0,Back-end Developer,12
1,Data Analyst,7
2,Data Engineer,4
3,Data Scientist,10
4,Database Administrator,2
5,Development Operations Engineer,3
6,Front-end Developer,8
7,Full-stack Developer,17
8,Mobile Developer,6
9,Network Administrator,1


Rows where Job Level is NaN and Current Role is not NaN:


,Job Level,Current Role
110,NaN,Software Test Engineer
513,NaN,Full-stack Developer
931,NaN,Data Scientist
1142,NaN,Front-end Developer
1433,NaN,Front-end Developer
...,...,...
6169,NaN,Data Analyst
6174,NaN,Software Engineer
6187,NaN,Back-end Developer
6202,NaN,Data Analyst


In [103]:
# Group by Current Role and Job Level, and count occurrences
role_level_counts = worker_responses_values_df.groupby(['Current Role', 'Job Level']).size().reset_index(name='Counts')

# Display the results
display(role_level_counts)


,Current Role,Job Level,Counts
0,Back-end Developer,Architect,1
1,Back-end Developer,Freelancer,1
2,Back-end Developer,Level 1 developer (junior),70
3,Back-end Developer,New grad,12
4,Back-end Developer,Principal engineer,2
...,...,...,...
81,Web Developer,Freelancer,2
82,Web Developer,Level 1 developer (junior),52
83,Web Developer,New grad,9
84,Web Developer,Principal engineer,2


#### Use Machine Learning to Populate NaN Job Level Records

In [105]:
# Use KNearestNeighbors to determine most likely Job Level based on age, gender, and current role

# Prepare the DataFrame for KNN training
train_df = worker_responses_values_df[worker_responses_values_df['Job Level'].notna()].copy()

# Create numeric mappings for Age and Gender for training
train_df.loc[:, 'Age Numeric'] = train_df['Age'].map(age_mapping)
train_df.loc[:, 'Gender Numeric'] = train_df['Gender'].map(gender_mapping)

# Encode Current Role using Label Encoding
le_role = LabelEncoder()
train_df.loc[:, 'Current Role Encoded'] = le_role.fit_transform(train_df['Current Role'])

# Prepare the feature set and target variable for training
X_train = train_df[['Age Numeric', 'Gender Numeric', 'Current Role Encoded']]
y_train = train_df['Job Level']

# Filter the rows with NaN Job Level for predictions
predict_df = worker_responses_values_df[worker_responses_values_df['Job Level'].isna()].copy()
predict_df.loc[:, 'Age Numeric'] = predict_df['Age'].map(age_mapping)
predict_df.loc[:, 'Gender Numeric'] = predict_df['Gender'].map(gender_mapping)
predict_df.loc[:, 'Current Role Encoded'] = le_role.transform(predict_df['Current Role'])

# Create the feature set for predictions
X_predict = predict_df[['Age Numeric', 'Gender Numeric', 'Current Role Encoded']]

# Initialize and fit KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict NaN Job Levels
predicted_levels = knn.predict(X_predict)

# Display the predictions before overwriting the original DataFrame
predict_df['Predicted Job Level'] = predicted_levels
print("Predicted Job Levels for records with NaN Job Level:")
display(predict_df[['Current Role', 'Predicted Job Level']])

Predicted Job Levels for records with NaN Job Level:


,Current Role,Predicted Job Level
110,Software Test Engineer,Level 1 developer (junior)
513,Full-stack Developer,Level 1 developer (junior)
931,Data Scientist,New grad
1142,Front-end Developer,Senior developer
1433,Front-end Developer,Level 1 developer (junior)
...,...,...
6169,Data Analyst,Level 1 developer (junior)
6174,Software Engineer,Senior developer
6187,Back-end Developer,Level 1 developer (junior)
6202,Data Analyst,Level 1 developer (junior)


In [107]:
#Update Job Level field with predictions and verify changes

# Update the Job Level in the original DataFrame where it was NaN
worker_responses_values_df.loc[worker_responses_values_df['Job Level'].isna(), 'Job Level'] = predicted_levels

# Verify that no records remain with NaN Job Level
remaining_nan_job_levels = worker_responses_values_df['Job Level'].isna().sum()
print(f"Number of records with NaN Job Level after updating: {remaining_nan_job_levels}")

# Display the cleaned DataFrame
display(worker_responses_values_df.head(5))

Number of records with NaN Job Level after updating: 0


,RespondentID,Age,Gender,Industry,Job Level,Current Role,q16HiringManager,q12JobCritPrefTechStack,q12JobCritCompMission,q12JobCritCompCulture,q12JobCritWorkLifeBal,q12JobCritCompensation,q12JobCritProximity,q12JobCritPerks,q12JobCritSmartPeopleTeam,q12JobCritImpactwithProduct,q12JobCritInterestProblems,q12JobCritFundingandValuation,q12JobCritStability,q12JobCritProfGrowth
7,6467198274,18 - 24 years old,Male,Education,New grad,NaN,No,NaN,NaN,Company culture,NaN,NaN,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,Professional growth and learning
44,6464279786,35 - 44 years old,Male,Technology,Principal engineer,Back-end Developer,No,NaN,NaN,NaN,Good work/life balance,Compensation,NaN,NaN,Smart people / team,NaN,NaN,NaN,NaN,NaN
48,6463324693,25 - 34 years old,Male,Technology,Senior developer,NaN,No,NaN,NaN,NaN,NaN,Compensation,NaN,Perks,NaN,NaN,NaN,NaN,NaN,Professional growth and learning
75,6463564885,25 - 34 years old,Male,Technology,Level 1 developer (junior),Software Engineer,No,NaN,NaN,NaN,NaN,Compensation,NaN,Perks,Smart people / team,NaN,NaN,NaN,NaN,Professional growth and learning
102,6464022754,25 - 34 years old,Male,Media,Level 1 developer (junior),Back-end Developer,No,NaN,NaN,Company culture,Good work/life balance,Compensation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Import Dataset 2:

In [5]:
# import zip file from Pew Research
file_handle, _ = urlretrieve("https://www.pewresearch.org/wp-content/uploads/sites/20/2019/04/2017-Pew-Research-Center-STEM-survey.zip")
zipfile = ZipFile(file_handle, "r")
zipfile.extractall("./data")
zipfile.close()

#### Import Dataset 3:

In [6]:
# scrape HTML file to extract tables
os.makedirs("data/ncses", exist_ok=True) # create new subfolder for tables scraped from website

url = "https://ncses.nsf.gov/pubs/nsb20212/participation-of-demographic-groups-in-stem"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser") # parse the html as a string

tables = soup.find_all("table")


# loop through each table to extract the title and data
for i in range(len(tables)):
    table = tables[i]

    # Extract the table's title attribute and clean it up for a filename
    title = table.get('title')  # Find the title attribute in the table

    if title:
        title = title.strip()  # Use the title if it exists
    else:
        title = f'table_{i}'  # Generic name if the table does not have a title
    
    # Clean title: replace spaces with hyphens and remove special characters except apostrophes
    title = title.replace("'", "")  # Remove apostrophes
    title = re.sub(r"[^\w\s]", "-", title)  # Replace special characters with hyphens
    title = title.replace(" ", "-")  # Replace spaces with hyphens

    # Truncate the title if it exceeds 50 characters for filename compatibility
    if len(title) > 50:
        title = title[:50]

    file_name = f"data/ncses/{title}.csv" # generate an empty csv for each table

    # convert each table to a pandas df
    df = pd.read_html(str(table))[0]

    # save the pandas df as a csv
    df.to_csv(file_name, index=False)

C:\Users\Michelle\AppData\Local\Temp\ipykernel_17388\2148890883.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Michelle\AppData\Local\Temp\ipykernel_17388\2148890883.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Michelle\AppData\Local\Temp\ipykernel_17388\2148890883.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Michelle\AppData\Local\Temp\ipykernel_17388\2148890883.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version

## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->

- https://it4063c.github.io/course-notes/working-with-data/data-sources for methods to import the various data types  
- https://www.kaggle.com/datasets/hackerrank/developer-survey-2018/data for the Kaggle dataset 
- https://www.pewresearch.org/social-trends/2018/01/09/women-and-men-in-stem-often-at-odds-over-workplace-equity/  for the link to the Pew Research Survey
- https://ncses.nsf.gov/pubs/nsb20212/participation-of-demographic-groups-in-stem for the html scraped dataset



In [1]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb

[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 7308 bytes to source.py
